First import Apache Spark and make an exploratory data analysis (EDA) on the play_by_play dataset:

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

spark = (SparkSession.builder
         .appName('eda_pbp')
         .master('local[4]')
         .config('spark.sql.adaptive.enabled', 'true')
         .getOrCreate())


In [5]:
# Path to pbp csv
print("Dataset description:")
df_bronze = spark.read.option('header', 'true').csv('../data/bronze/play_by_play/play_by_play_2023.csv')
print(f"Rows: {df_bronze.count():,}")
df_bronze.printSchema()

Dataset description:
Rows: 49,665
root
 |-- play_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- old_game_id: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- season_type: string (nullable = true)
 |-- week: string (nullable = true)
 |-- posteam: string (nullable = true)
 |-- posteam_type: string (nullable = true)
 |-- defteam: string (nullable = true)
 |-- side_of_field: string (nullable = true)
 |-- yardline_100: string (nullable = true)
 |-- game_date: string (nullable = true)
 |-- quarter_seconds_remaining: string (nullable = true)
 |-- half_seconds_remaining: string (nullable = true)
 |-- game_seconds_remaining: string (nullable = true)
 |-- game_half: string (nullable = true)
 |-- quarter_end: string (nullable = true)
 |-- drive: string (nullable = true)
 |-- sp: string (nullable = true)
 |-- qtr: string (nullable = true)
 |-- down: string (nullable = true)
 |-- goal_to_go: string (nullable = 

Important columns for analysis:
- play_id
- game_id
- game_date
- season
- week
- home_team
- away_team
- play_type
- yards_gained
- posteam (offense)
- defteam (defense)
- epa (Expected Points Added, measures how much a single play improves or worsens a team's scoring chances, calculated by subtracting the Expected Points (EP) before a play from the EP after the play)
- wp (win probability)

In [5]:
important_cols = ['play_id', 'game_id', 'game_date', 'season', 'week', 'home_team', 'away_team', 'posteam', 'defteam',
                  'play_type', 'yards_gained', 'epa', 'wp']
df_bronze.select(important_cols).describe()

DataFrame[summary: string, play_id: string, game_id: string, game_date: string, season: string, week: string, home_team: string, away_team: string, posteam: string, defteam: string, play_type: string, yards_gained: string, epa: string, wp: string]

As we can see, the dataframe stored every column as string, this will be important for future transformations

In [16]:
from pyspark.sql.functions import col, count, when, lit

# Count nulls per column
null_counts_df = df_bronze.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_bronze.columns
])

# Legible print
null_counts_long = null_counts_df.selectExpr(
    "stack({}, {}) as (column_name, null_count)".format(
        len(df_bronze.columns),
        ",".join([f"'{c}', `{c}`" for c in df_bronze.columns])
    )
)

null_counts_long.filter(col("null_count") > 0) \
                .orderBy(col("null_count").desc()) \
                .show(null_counts_long.count(), truncate=False)

+------------------------------------+----------+
|column_name                         |null_count|
+------------------------------------+----------+
|lateral_sack_player_id              |49665     |
|lateral_sack_player_name            |49665     |
|lateral_punt_returner_player_id     |49665     |
|lateral_punt_returner_player_name   |49665     |
|tackle_for_loss_2_player_id         |49665     |
|tackle_for_loss_2_player_name       |49665     |
|assist_tackle_3_player_id           |49665     |
|assist_tackle_3_player_name         |49665     |
|assist_tackle_3_team                |49665     |
|assist_tackle_4_player_id           |49665     |
|assist_tackle_4_player_name         |49665     |
|assist_tackle_4_team                |49665     |
|tackle_with_assist_2_player_id      |49665     |
|tackle_with_assist_2_player_name    |49665     |
|tackle_with_assist_2_team           |49665     |
|st_play_type                        |49665     |
|end_yard_line                       |49665     |
